In [75]:
import gcsfs
import zarr
import xarray as xr

# Set up the Google Cloud Storage filesystem
fs = gcsfs.GCSFileSystem()

# Path to the Zarr store in GCS
# gcs_path = 'gs://weatherbench2/datasets/hres_t0/2016-2022-6h-1440x721.zarr'   # I am using this
# gcs_path = 'gs://weatherbench2/datasets/hres/2016-2022-12h-6h-0p25deg-chunk-1.zarr'
# gcs_path = 'gs://weatherbench2/datasets/hres/2016-2022-0012-1440x721.zarr'

# gcs_path = 'gs://weatherbench2/datasets/graphcast_hres_init/2020/date_range_2019-11-16_2021-02-01_12_hours_derived.zarr'
gcs_path = 'gs://weatherbench2/datasets/graphcast/2018/date_range_2017-11-16_2019-02-01_12_hours_derived.zarr'

# Try opening the Zarr store with and without consolidated metadata
try:
    # Attempt to open as a consolidated Zarr store
    store = gcsfs.GCSMap(gcs_path, gcs=fs)
    zarr_dataset = zarr.open_consolidated(store, mode='r')
except KeyError:
    # If that fails, open as a non-consolidated Zarr store
    zarr_dataset = zarr.open_group(store, mode='r')

# Load the Zarr dataset into an xarray Dataset
ds = xr.open_zarr(store)

# Check available variables in the dataset
print(ds)

# Select the wind_speed variable if it exists

<xarray.Dataset>
Dimensions:                   (time: 884, prediction_timedelta: 40, lat: 721,
                               lon: 1440, level: 37)
Coordinates:
  * lat                       (lat) float32 -90.0 -89.75 -89.5 ... 89.75 90.0
  * level                     (level) int64 1 2 3 5 7 ... 900 925 950 975 1000
  * lon                       (lon) float32 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * prediction_timedelta      (prediction_timedelta) timedelta64[ns] 06:00:00...
  * time                      (time) datetime64[ns] 2017-11-16 ... 2019-01-31...
Data variables: (12/14)
    10m_u_component_of_wind   (time, prediction_timedelta, lat, lon) float32 dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind   (time, prediction_timedelta, lat, lon) float32 dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>
    10m_wind_speed            (time, prediction_timedelta, lat, lon) float32 dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>
    2m_temperat

In [78]:
wind_speed = ds['10m_wind_speed']

In [80]:
wind_speed.sel(time='2018-04-01T00:00:00')

<xarray.DataArray '10m_wind_speed' (prediction_timedelta: 40, lat: 721,
                                    lon: 1440)>
dask.array<getitem, shape=(40, 721, 1440), dtype=float32, chunksize=(1, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat                   (lat) float32 -90.0 -89.75 -89.5 ... 89.5 89.75 90.0
  * lon                   (lon) float32 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * prediction_timedelta  (prediction_timedelta) timedelta64[ns] 0 days 06:00...
    time                  datetime64[ns] 2018-04-01

In [83]:
wind_speed.sel(time='2018-04-04T00:00:00').to_netcdf('extreme_wind_2018_california_storm_surface.nc')

In [39]:
lat_min = 47.0
lat_max = 56.0
lon_min = 184.75
lon_max = 195.0

# Filter the dataset for Germany's boundaries
# wind_speed_germany = wind_speed.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max), level=1000)
weather_germany = ds[['u_component_of_wind', 'v_component_of_wind', 'wind_speed', 'temperature', 'specific_humidity']].sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max), level=1000)



In [41]:
weather_germany.to_netcdf('weather_germany_2016-2022.nc')

In [42]:
weather_germany

<xarray.Dataset>
Dimensions:              (time: 10268, latitude: 37, longitude: 42)
Coordinates:
  * latitude             (latitude) float32 47.0 47.25 47.5 ... 55.5 55.75 56.0
    level                int32 1000
  * longitude            (longitude) float32 184.8 185.0 185.2 ... 194.8 195.0
  * time                 (time) datetime64[ns] 2016-01-01 ... 2023-01-10T18:0...
Data variables:
    u_component_of_wind  (time, latitude, longitude) float32 dask.array<chunksize=(1, 37, 42), meta=np.ndarray>
    v_component_of_wind  (time, latitude, longitude) float32 dask.array<chunksize=(1, 37, 42), meta=np.ndarray>
    wind_speed           (time, latitude, longitude) float32 dask.array<chunksize=(1, 37, 42), meta=np.ndarray>
    temperature          (time, latitude, longitude) float32 dask.array<chunksize=(1, 37, 42), meta=np.ndarray>
    specific_humidity    (time, latitude, longitude) float32 dask.array<chunksize=(1, 37, 42), meta=np.ndarray>

In [53]:
w = xr.open_dataset("weather-germany-2016-2022.nc")

In [52]:
w['specific_humidity'].to_netcdf('specific_humidity_2016-2022.nc')

In [5]:
wind_speed_germany.to_netcdf('wind_speed_germany_2016-2022.nc')

In [23]:
wind_speed.sel(latitude=55, longitude=190, time="2022-12-31", level=1000).to_numpy()

array([3.265245 , 3.801176 , 5.4927115, 7.391374 ], dtype=float32)